# Average Episode BPM
> Tracking average episode BPM over time.

- toc: true 
- badges: true
- comments: false
- categories: [asot]
- image: images/chart-preview.png

In [3]:
#hide
import os
import yaml
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials

with open('spotipy_credentials.yaml', 'r') as spotipy_credentials_file:
    credentials = yaml.safe_load(spotipy_credentials_file)
    os.environ["SPOTIPY_CLIENT_ID"] = credentials['spotipy_credentials']['spotipy_client_id']
    os.environ["SPOTIPY_CLIENT_SECRET"] = credentials['spotipy_credentials']['spotipi_client_seret']

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

## Introduction


Nice! Looks like the "`tempo`", or BPM, of this track is around 133. Let's continue.

Conveniently, the entire back catalogue of A State of Trance - 950+ episodes - has been uploaded to Spotify under the artist ["Armin van Buuren ASOT Radio"](https://open.spotify.com/artist/25mFVpuABa9GkGcj9eOPce). [Spotipy's `artist_albums` method](https://spotipy.readthedocs.io/en/2.12.0/?highlight=audio_features#spotipy.client.Spotify.artist_albums) can list them for us, courtesy [`spotipy/examples/artist_albums.py`](https://github.com/plamere/spotipy/blob/2584d8cf5675ce877f773112a76d42fe36f8a1d1/examples/artist_albums.py#L29-L42):

In [11]:
"""
Artist: Armin van Buuren ASOT Radio
Artist link: https://open.spotify.com/artist/25mFVpuABa9GkGcj9eOPce
Artist ID: 25mFVpuABa9GkGcj9eOPce
"""

asot_radio_id = '25mFVpuABa9GkGcj9eOPce'


episodes = []
results = sp.artist_albums(asot_radio_id, album_type='album')
episodes.extend(results['items'])
while results['next']:
    results = sp.next(results)
    episodes.extend(results['items'])
seen = set()  # to avoid dups
for episode in episodes:
    name = episode['name']
    if name not in seen:
        seen.add(name)

episodes = episodes[::-1]

Cool, our dictionary `episodes` should now contain every episode of A State of Trance! Let's take a quick look..

In [12]:
for episode in episodes[:10]:
    print(episode['name'])

A State Of Trance Episode 001
A State Of Trance Episode 003
A State Of Trance Episode 004
A State Of Trance Episode 005
A State Of Trance Episode 007
A State Of Trance Episode 008
A State Of Trance Episode 009
A State Of Trance Episode 010
A State Of Trance Episode 012
A State Of Trance Episode 015


Hm, aren't we missing a few?

In [6]:
episode_tracks = sp.album_tracks(episodes[1]['uri'])
print(json.dumps(episode_tracks, indent=2))

{
  "href": "https://api.spotify.com/v1/albums/2RLKYKweji0JBAi2OtxOCt/tracks?offset=0&limit=50",
  "items": [
    {
      "artists": [
        {
          "external_urls": {
            "spotify": "https://open.spotify.com/artist/0SfsnGyD8FpIN4U4WCkBZ5"
          },
          "href": "https://api.spotify.com/v1/artists/0SfsnGyD8FpIN4U4WCkBZ5",
          "id": "0SfsnGyD8FpIN4U4WCkBZ5",
          "name": "Armin van Buuren",
          "type": "artist",
          "uri": "spotify:artist:0SfsnGyD8FpIN4U4WCkBZ5"
        }
      ],
      "available_markets": [
        "AD",
        "AE",
        "AR",
        "AT",
        "AU",
        "BE",
        "BG",
        "BH",
        "BO",
        "BR",
        "CA",
        "CH",
        "CL",
        "CO",
        "CR",
        "CY",
        "CZ",
        "DE",
        "DK",
        "DO",
        "DZ",
        "EC",
        "EE",
        "EG",
        "ES",
        "FI",
        "FR",
        "GB",
        "GR",
        "GT",
        "HK",
       

In [14]:
#hide
audio_features = sp.audio_features(episode_tracks['items'][2]['uri'])
print(json.dumps(audio_features, indent=2))

[
  {
    "danceability": 0.588,
    "energy": 0.965,
    "key": 11,
    "loudness": -11.546,
    "mode": 0,
    "speechiness": 0.0587,
    "acousticness": 0.000847,
    "instrumentalness": 0.936,
    "liveness": 0.39,
    "valence": 0.45,
    "tempo": 133.053,
    "type": "audio_features",
    "id": "6OqkwRDlfdkMcs7P3TDn5Z",
    "uri": "spotify:track:6OqkwRDlfdkMcs7P3TDn5Z",
    "track_href": "https://api.spotify.com/v1/tracks/6OqkwRDlfdkMcs7P3TDn5Z",
    "analysis_url": "https://api.spotify.com/v1/audio-analysis/6OqkwRDlfdkMcs7P3TDn5Z",
    "duration_ms": 437156,
    "time_signature": 4
  }
]


In [15]:
import altair as alt
import numpy as np
import pandas as pd

In [16]:
bpm = []
for track in episode_tracks['items']:
    bpm.append(sp.audio_features(track['uri'])[0]['tempo'])

x = np.arange(len(episode_tracks['items']))   

source = pd.DataFrame({
  'track': x,
  'bpm': np.array(bpm)
})

alt.Chart(source).mark_line().encode(
    x='track',
    y='bpm'
)

alt.Chart(...)

In [17]:
#hide
episode_avg_bpm = []

for episode in episodes[:10]:
    try:
        episode_bpm = 0
        tracks_counted = 0
        for track in sp.album_tracks(episode['uri'])['items']:
            if "A State Of Trance" in track['name']:
                continue
            else:
                episode_bpm += sp.audio_features(track['uri'])[0]['tempo']
                tracks_counted += 1
        avg = episode_bpm/tracks_counted
        print(episode['name'], avg)
        episode_avg_bpm.append(avg)
    except:
        pass

A State Of Trance Episode 001 138.86957142857142
A State Of Trance Episode 003 135.295
A State Of Trance Episode 004 137.3078
A State Of Trance Episode 005 137.122875
A State Of Trance Episode 007 137.459125
A State Of Trance Episode 008 136.409
A State Of Trance Episode 009 138.58033333333333
A State Of Trance Episode 010 137.5172
A State Of Trance Episode 012 138.5285
A State Of Trance Episode 015 138.09266666666667


In [18]:
x = np.arange(len(episode_avg_bpm))   

source = pd.DataFrame({
  'episode': x,
  'avg bpm': np.array(episode_avg_bpm)
})

alt.Chart(source).mark_line().encode(
    alt.X('episode'),
    alt.Y('avg bpm', scale=alt.Scale(domain=(120, 150))),
).properties(
    title="A State of Trance - Average BPM of episode"
)

alt.Chart(...)

In [19]:
episode_bpm = 0
tracks_counted = 0
for track in sp.album_tracks('56qM5Y21wbvCW9l5GiAiaV')['items']:
    if "A State Of Trance" in track['name']:
        continue
    else:
        print(track['name'], ' - ', sp.audio_features(track['uri'])[0]['tempo'])
        episode_bpm += sp.audio_features(track['uri'])[0]['tempo']
        tracks_counted += 1
print(episode_bpm/tracks_counted)

Cloudwalking [ASOT 057] - Beatpusher Remix  -  141.61
Ligaya [ASOT 057] - Original Mix  -  141.784
Obsession [ASOT 057] - Original Mix  -  136.377
Gatex [ASOT 057] - Original Mix  -  184.848
Positron [ASOT 057] - Marco V Remix  -  139.264
148.7766


In [20]:
x = np.arange(len(episode_avg_bpm))

source = pd.DataFrame({
  'episode': x,
  'avg bpm': np.array(episode_avg_bpm)
})

source['138'] = 138

base = alt.Chart(source).mark_line().encode(
    alt.X('episode'),
    alt.Y('avg bpm', scale=alt.Scale(domain=(120, 150))),
).properties(
    title="A State of Trance - Average BPM of episode"
)

rule = alt.Chart(source).mark_rule(color='red').encode(
    y='138'
)

base + rule

alt.LayerChart(...)